<div style="text-align: right" align="right"><i>Peter Norvig<br>Decembers 2016–2021</i></div>

# Advent of Code Utilities

Stuff I might need for [Advent of Code](https://adventofcode.com). First, some imports that I have used in past AoC years:

In [1]:
from collections import Counter, defaultdict, namedtuple, deque, abc
from dataclasses import dataclass
from itertools   import permutations, combinations, cycle, chain
from itertools   import count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, floor, factorial, gcd, log, log2, log10, sqrt, inf
from functools   import reduce

import matplotlib.pyplot as plt
import time
import heapq
import string
import functools
import pathlib
import re

# Daily Input Parsing

Each day's work will consist of three tasks, denoted by three sections in the notebook:
- **Input**: Parse the day's input file.  I will  use the function `parse(day, parser, sep)`, which:
   - Reads the input file for `day`.
   - Breaks the file into a sequence of *items* separated by `sep` (default newline).
   - Applies `parser` to each item and returns the results as a tuple.
       - Useful parser functions include `ints`, `digits`, `atoms`, `words`, and the built-ins `int` and `str`.
   - Prints the first few input lines and output records. This is useful to me as a debugging tool, and to the reader.
- **Part 1**: Understand the day's instructions and:
   - Write code to compute the answer to Part 1.
   - Once I have computed the  answer and submitted it to the AoC site to verify it is correct, I  record it with the `answer` function.
- **Part 2**: Repeat the above steps for Part 2.
- Occasionally I'll introduce a **Part 3** where I explore beyond the official instructions.

Here is `parse` and some helper functions for it:

In [2]:
current_year = 2022  # Subdirectory name for input files
lines = '\n'         # For inputs where each record is a line
paragraphs = '\n\n'  # For inputs where each record is a paragraph 

def parse(day_or_text:Union[int, str], parser:Callable=str, sep:Callable=lines, show=6) -> tuple:
    """Split the input text into items separated by `sep`, and apply `parser` to each.
    The first argument is either the text itself, or the day number of a text file."""
    text = get_text(day_or_text)
    print_parse_items('Puzzle input', text.splitlines(), show, 'line')
    records = mapt(parser, text.rstrip().split(sep))
    if parser != str:
        print_parse_items('Parsed representation', records, show, f'{type(records[0]).__name__}')
    return records

def get_text(day_or_text:Union[int, str]) -> str:
    """The text used as input to the puzzle: either a string or the day number of a file."""
    if isinstance(day_or_text, int):
        return pathlib.Path(f'AOC/{current_year}/input{day_or_text}.txt').read_text()
    else:
        return day_or_text

def print_parse_items(source, items, show:int, name:str, sep="─"*100):
    """Print verbose output from `parse` for lines or records."""
    if not show:
        return
    count = f'1 {name}' if len(items) == 1 else f'{len(items)} {name}s'
    for line in (sep, f'{source} ➜ {count}:', sep, *items[:show]):
        print(truncate(line))
    if show < len(items):
        print('...')
        
def truncate(object, width=100) -> str:
    """Use elipsis to truncate `str(object)` to `width` characters, if necessary."""
    string = str(object)
    return string if len(string) <= width else string[:width-4] + ' ...'

def parse_sections(specs: Iterable) -> Callable:
    """Return a parser that uses the first spec to parse the first section, the second for second, etc.
    Each spec is either parser or [parser, sep]."""
    specs = ([spec] if callable(spec) else spec for spec in specs)
    fns = ((lambda section: parse(section, *spec, show=0)) for spec in specs)
    return lambda section: next(fns)(section)

In [3]:
## Functions that can be used by `parse`

Char = str # Intended as the type of a one-character string
Atom = Union[str, float, int] # The type of a string or number

def ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'-?[0-9]+', text))

def positive_ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'[0-9]+', text))

def digits(text: str) -> Tuple[int]:
    """A tuple of all the digits in text (as ints 0–9), ignoring non-digit characters."""
    return mapt(int, re.findall(r'[0-9]', text))

def words(text: str) -> Tuple[str]:
    """A tuple of all the alphabetic words in text, ignoring non-letters."""
    return tuple(re.findall(r'[a-zA-Z]+', text))

def atoms(text: str) -> Tuple[Atom]:
    """A tuple of all the atoms (numbers or identifiers) in text. Skip punctuation."""
    return mapt(atom, re.findall(r'[+-]?\d+\.?\d*|\w+', text))

def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if x.is_integer() else x
    except ValueError:
        return text
    
def mapt(function: Callable, sequence) -> tuple:
    """`map`, with the result as a tuple."""
    return tuple(map(function, sequence))

In [4]:
## TESTS

assert parse("hello\nworld", show=0) == ('hello', 'world')
assert parse("123\nabc7", digits, show=0) == ((1, 2, 3), (7,))
assert truncate('hello world', 99) == 'hello world'
assert truncate('hello world', 8)  == 'hell ...'

assert         atoms('hello, cruel_world! 24-7') == ('hello', 'cruel_world', 24, -7)
assert         words('hello, cruel_world! 24-7') == ('hello', 'cruel', 'world')
assert        digits('hello, cruel_world! 24-7') == (2, 4, 7)
assert          ints('hello, cruel_world! 24-7') == (24, -7)
assert positive_ints('hello, cruel_world! 24-7') == (24, 7)

# Daily Answers

Here is the `answer` function, which gives verification of a correct computation (or an error message for an incorrect computation), times how long the computation took, ans stores the result in the dict `answers`.

In [5]:
# `answers` is a dict of {puzzle_number_id: message_about_results}
answers = {} 

def answer(puzzle, correct, code: callable) -> str:
    """Verify that calling `code` computes the `correct` answer for `puzzle`. 
    Record results in the dict `answers`. Prints execution time."""
    def pretty(x): return f'{x:,d}' if is_int(x) else truncate(x)
    start = time.time()
    got   = code()
    dt    = time.time() - start
    ans   = pretty(got)
    msg = f'{dt:5.3f} seconds for ' + (
        f'correct answer: {ans}' if (got == correct) else
        f'WRONG!! ANSWER: {ans}; EXPECTED {pretty(correct)}')
    answers[puzzle] = msg
    return msg

# Additional  utility functions 

All of the following have been used in solutions to multiple puzzles in the past, so I pulled them all in here:

In [6]:
def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

class multimap(defaultdict):
    """A mapping of {key: [val1, val2, ...]}."""
    def __init__(self, pairs: Iterable[tuple], symmetric=False):
        """Given (key, val) pairs, return {key: [val, ...], ...}.
        If `symmetric` is True, treat (key, val) as (key, val) plus (val, key)."""
        self.default_factory = list
        for (key, val) in pairs:
            self[key].append(val)
            if symmetric:
                self[val].append(key)

def prod(numbers) -> float: # Will be math.prod in Python 3.8
    """The product formed by multiplying `numbers` together."""
    result = 1
    for x in numbers:
        result *= x
    return result

def total(counter: Counter) -> int: 
    """The sum of all the counts in a Counter."""
    return sum(counter.values())

def minmax(numbers) -> Tuple[int, int]:
    """A tuple of the (minimum, maximum) of numbers."""
    numbers = list(numbers)
    return min(numbers), max(numbers)

def first(iterable) -> Optional[object]: 
    """The first element in an iterable, or None."""
    return next(iter(iterable), None)

def T(matrix: Sequence[Sequence]) -> List[Tuple]:
    """The transpose of a matrix: T([(1,2,3), (4,5,6)]) == [(1,4), (2,5), (3,6)]"""
    return list(zip(*matrix))

def cover(*integers) -> range:
    """A `range` that covers all the given integers, and any in between them.
    cover(lo, hi) is a an inclusive (or closed) range, equal to range(lo, hi + 1)."""
    return range(min(integers), max(integers) + 1)

def the(sequence) -> object:
    """Return the one item in a sequence. Raise error if not exactly one."""
    items = list(sequence)
    if not len(items) == 1:
        raise ValueError(f'Expected exactly one item in the sequence {items}')
    return items[0]

def split_at(sequence, i) -> Tuple[Sequence, Sequence]:
    """The sequence split into two pieces: (before position i, and i-and-after)."""
    return sequence[:i], sequence[i:]

def batched(data, n) -> list:
    "Batch data into lists of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    return [data[i:i+n] for i in range(0, len(data), n)]

def sliding_window(sequence, n) -> Iterable[Sequence]:
    """All length-n subsequences of sequence."""
    return (sequence[i:i+n] for i in range(len(sequence) + 1 - n))

def ignore(*args) -> None: "Just return None."; return None

def is_int(x) -> bool: "Is x an int?"; return isinstance(x, int)  

def sign(x) -> int: "0, +1, or -1"; return (0 if x == 0 else +1 if x > 0 else -1)

def append(sequences) -> Sequence: "Append sequences into a list"; return list(flatten(sequences))

def union(sets) -> set: "Union of several sets"; return set().union(*sets)

def intersection(sets):
    "Intersection of several sets."
    first, *rest = sets
    return set(first).intersection(*rest)

def square_plot(points, marker='o', size=12, extra=None, **kwds):
    """Plot `points` in a square of given `size`, with no axis labels.
    Calls `extra()` to do more plt.* stuff if defined."""
    plt.figure(figsize=(size, size))
    plt.plot(*T(points), marker, **kwds)
    if extra: extra()
    plt.axis('square'); plt.axis('off'); plt.gca().invert_yaxis()
    
def clock_mod(i, m) -> int:
    """i % m, but replace a result of 0 with m"""
    # This is like a clock, where 24 mod 12 is 12, not 0.
    return (i % m) or m

flatten = chain.from_iterable # Yield items from each sequence in turn
cat     = ''.join
cache   = functools.lru_cache(None)

In [17]:
## TESTS

assert quantify(words('This is a test'), str.islower) == 3
assert mapt(first, words('This is a test')) == ('T', 'i', 'a', 't')
assert multimap(((i % 3), i) for i in range(9)) == {0: [0, 3, 6], 1: [1, 4, 7], 2: [2, 5, 8]}
assert prod([2, 3, 5]) == 30
assert total(Counter('hello, world')) == 12
assert cover(3, 1, 4, 1, 5) == range(1, 6)
assert minmax([3, 1, 4, 1, 5, 9]) == (1, 9)
assert first('abc') == 'a'
assert T([(1, 2, 3), (4, 5, 6)]) == [(1, 4), (2, 5), (3, 6)]
assert the({1}) == 1
assert split_at('hello, world', 6) == ('hello,', ' world')
assert batched('abcdefghi', 3) == ['abc', 'def', 'ghi']
assert list(sliding_window('abcdefghi', 3)) == ['abc', 'bcd', 'cde', 'def', 'efg', 'fgh', 'ghi']
assert is_int(-42) and not is_int('one')
assert sign(-42) == -1 and sign(0) == 0 and sign(42) == +1
assert append(([1, 2], [3, 4], [5, 6])) == [1, 2, 3, 4, 5, 6]
assert union([{1, 2}, {3, 4}, {5, 6}]) == {1, 2, 3, 4, 5, 6}
assert intersection([{1, 2, 3}, {2, 3, 4}, {2, 4, 6, 8}]) == {2}
assert clock_mod(24, 12) == 12 and 24 % 12 == 0
assert list(flatten(['abc', 'def', '123'])) == ['a', 'b', 'c', 'd', 'e', 'f', '1', '2', '3']
assert cat(['hello', 'world']) == 'helloworld'

# Points on a Grid

Many puzzles seem to involve a two-dimensional rectangular grid with integer coordinates. First we'll define the two-dimensional `Point`, then the `Grid`.

In [7]:
Point = Tuple[int, int] # (x, y) points on a grid

def X_(point) -> int: "X coordinate"; return point[0]
def Y_(point) -> int: "Y coordinate"; return point[1]

def distance(p: Point, q: Point) -> float:
    """Distance between two points."""
    dx, dy = abs(X_(p) - X_(q)), abs(Y_(p) - Y_(q))
    return dx + dy if dx == 0 or dy == 0 else (dx ** 2 + dy ** 2) ** 0.5

def manhatten_distance(p: Point, q: Point) -> int:
    """Distance along grid lines between two points."""
    return sum(abs(pi - qi) for pi, qi in zip(p, q))

def add(p: Point, q: Point) -> Point:
    """Add two points."""
    return (X_(p) + X_(q), Y_(p) + Y_(q))

def sub(p: Point, q: Point) -> Point:
    """Subtract point q from point p."""
    return (X_(p) - X_(q), Y_(p) - Y_(q))

directions4 = North, South, East, West = ((0, -1), (0, 1), (1, 0), (-1, 0))
directions8 = directions4 + ((1, 1), (1, -1), (-1, 1), (-1, -1))

In [16]:
## TESTS

p, q = (0, 3), (4, 0)
assert Y_(p) == 3 and X_(q) == 4
assert distance(p, q) == 5
assert manhatten_distance(p, q) == 7
assert add(p, q) == (4, 3)
assert sub(p, q) == (-4, 3)
assert add(North, South) == (0,0)

In [8]:
class Grid(dict):
    """A 2D grid, implemented as a mapping of {(x, y): cell_contents}."""
    def __init__(self, mapping_or_rows, directions=directions4):
        """Initialize with either (e.g.) `Grid({(0, 0): 1, (1, 0): 2, ...})`, or
        `Grid([(1, 2, 3), (4, 5, 6)])."""
        self.update(mapping_or_rows if isinstance(mapping_or_rows, abc.Mapping) else
                    {(x, y): val 
                     for y, row in enumerate(mapping_or_rows) 
                     for x, val in enumerate(row)})
        self.width  = max(map(X_, self)) + 1
        self.height = max(map(Y_, self)) + 1
        self.directions = directions
        
    def copy(self): return Grid(self, directions=self.directions)
    
    def neighbors(self, point) -> List[Point]:
        """Points on the grid that neighbor `point`."""
        return [add(point, Δ) for Δ in self.directions if add(point, Δ) in self]
    
    def to_rows(self, default='.') -> List[List[object]]:
        """The contents of the grid in a rectangular list of lists."""
        return [[self.get((x, y), default) for x in range(self.width)]
                for y in range(self.height)]
    
    def to_picture(self, sep='', default='.') -> str:
        """The contents of the grid as a picture."""
        return '\n'.join(map(cat, self.to_rows(default)))

# A* Search

Many puzzles involve searching over a branching tree of possibilities. For many puzzles, an ad-hoc solution is fine. But when there is a larger search space, it is useful to have a pre-defined efficient best-first search algorithm, and in particular an A* search, which incorporates a heuristic function to estimate the remaining distance to the goal.  This is a somewhat heavy-weight approach, as it requires the solver to define a subclass of `SearchProblem`.

In [18]:
def A_star_search(problem, h=None):
    """Search nodes with minimum f(n) = path_cost(n) + h(n) value first."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: n.path_cost + h(n))

def best_first_search(problem, f) -> 'Node':
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return search_failure

In [10]:
class SearchProblem:
    """The abstract class for a search problem. A new domain subclasses this,
    overriding `actions` and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def __str__(self):
        return '{}({!r}, {!r})'.format(type(self).__name__, self.initial, self.goal)
    
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): return action # Simplest case: action is result state
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0 # Never overestimate!
    
class GridProblem(SearchProblem):
    """Problem for searching a grid from a start to a goal location.
    A state is just an (x, y) location in the grid."""
    def actions(self, loc):           return self.grid.neighbors(loc)
    def result(self, loc1, loc2):     return loc2
    def action_cost(self, s1, a, s2): return self.grid[s2]
    def h(self, node): return manhatten_distance(node.state, self.goal) 

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self):      return f'Node({self.state})'
    def __len__(self):       return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
search_failure = Node('failure', path_cost=inf) # Indicates an algorithm couldn't find a solution.
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s2 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s2)
        yield Node(s2, node, action, cost)
        
def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]

def path_states(node):
    "The sequence of states to get to this node."
    if node in (search_failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [11]:
class PriorityQueue:
    """A queue in which the item with minimum key(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queue."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)